In [12]:
## 分析互作cliques
import networkx as nx
import pandas as pd
def find_cliques(input_f, output_f):
    graphics = nx.read_edgelist(input_f)
    cliques = nx.find_cliques(graphics)
    df_tmp = pd.DataFrame(columns=['cliques_size'])
    for i in cliques:
        #print(i)
        df_tmp.loc[df_tmp.shape[0]] = [len(i)]
    df_tmp['number'] = [1]*df_tmp.shape[0]
    df_output = df_tmp.groupby(by=['cliques_size'], as_index=False)['number'].agg(sum)
    df_output.to_csv(output_f, sep='\t', index=False)

In [ ]:
## 分析Pore-C 酶切片段互作cliques
find_cliques('F:/TM-1_leaf_3D-genome/Pore-C/cliques/1Kb/Pore-C_frag_1K_node_3_005.txt', "F:/TM-1_leaf_3D-genome/Pore-C/cliques/1Kb/Pore-C_frag_1K_node_3_005_cliques.txt")

In [16]:
## 分析Pore-C 酶切片段互作cliques
find_cliques('F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/Pore_C_TAD_50_005_node1.txt', "F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/Pore_C_TAD_50_005_cliques.txt")

In [105]:
## 分析Hi-C TAD 片段互作cliques
find_cliques('F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/Pore_C_TAD_20_0005_node1.txt', "F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/Pore_C_TAD_20_0005_cliques.txt")

242477


In [ ]:
## 分析Hi-C TAD 片段互作cliques
find_cliques('F:/TM-1_leaf_3D-genome/HiC/TAD/Cliques/method1/Hi-C_TAD_100_0005_node1.txt', "F:/TM-1_leaf_3D-genome/HiC/TAD/Cliques/Hi-C_TAD_100_0005_cliques.txt")

In [4]:
## 分析Hi-C TAD 片段互作 cliques 方法2
find_cliques('F:/TM-1_leaf_3D-genome/Proe-C_HiC/TAD/Cliques/method2/TM1_leaf_TAD_node1.txt', "F:/TM-1_leaf_3D-genome/Proe-C_HiC/TAD/Cliques/method2/TAD_loop_interactions_cliques.txt")


In [8]:
## 分析Hi-C 酶切片段互作 cliques
find_cliques('F:/TM-1_leaf_3D-genome/HiC/fragment/cliques/TM1_leaf_HiC_fragment_interaction_3_005_node1.txt', "F:/TM-1_leaf_3D-genome/HiC/fragment/cliques/TM1_leaf_HiC_fragment_interaction_3_005_cliques.txt")

In [46]:
find_cliques('F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/Pore_C_TAD_50_005_node_A05_cliques_edg.txt', "F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/Pore_C_TAD_50_005_node_A05_cliques.txt")

In [ ]:
## 将TAD 互作的片段 进行权重标准化
import pandas as pd
import sys
TAD = sys.argv[1]
TAD_I = sys.argv[2]
out_f = sys.argv[3]
df_tad = pd.read_csv(TAD, sep="\t")
df_tadi = pd.read_csv(TAD_I, sep="\t", names=['source', 'target', 'number'])
df_tad['tags'] = (df_tad['e'] - df_tad['s'])/2 + df_tad['s']
df_tad['tags'] = df_tad['tags'].astype(chr)
df_tad['size'] = df_tad['e'] - df_tad['s']
df_tad['tags2'] = df_tad['chr'] + "-" + df_tad['tags'] 
df_tad_out = df_tad[['tags2', 'size']].copy()
df_tad_out.columns = ['source', 'source_size']
df_tad_tmp = pd.merge(df_tadi, df_tad_out, how='inner', on=['source'])
df_tad_out.columns = ['target', 'target_size']
df_tad_new = pd.merge(df_tad_out, df_tad_tmp, how='inner', on=['target'])
df_tad_new['weight'] = (df_tad_new['number'].map(float)/(df_tad_new['source_size'].map(float)*df_tad_new['target_size'].map(float)))*100
df_tad_new.to_csv(out_f, sep='\t', index=False)



In [ ]:
## analysis TAD-TAD interaction 
import pandas as pd
import sys
tad_f = sys.argv[1]
loop_f = sys.argv[2]
output_f = sys.argv[3]
df_tad = pd.read_csv(tad_f, sep='\t', names=['c', 's', 'e'])
df_loop = pd.read_csv(loop_f, sep='\t', names=['c1', 'site1', 'c2', 'site2', 'numbers'])
## tad tags
def tad_tad_interaction(df_tad, df_loop):
    df_tad['tags'] = df_tad['c'] + "-" + df_tad['s'].map(str)
    df_tmp1 = pd.DataFrame()
    for index, row in df_tad.iterrows():
        c, s, e, t = row['c'], row['s'], row['e'], row['tags']
        df_loop_tmp = df_loop.query("c1==@c & site1>=@s & site2<=@e")
        df_loop_tmp['tags1'] = [t]*df_loop_tmp.shape[0]
        df_tmp1 = df_tmp1.append(df_loop_tmp, ignore_index=True)
    df_tmp2 = pd.DataFrame()
    for index, row in df_tad.iterrows():
        c, s, e, t = row['c'], row['s'], row['e'], row['tags']
        df_loop_tmp2 = df_tmp1.query("c2==@c & site2>=@s & site2<=@e")
        df_loop_tmp2['tags2'] = [t]*df_loop_tmp2.shape[0]
        df_tmp2 = df_tmp2.append(df_tmp2, ignore_index=True)
    df_out_tmp = df_tmp2[['tags1', 'tags2', 'numbers']]
    df_out = df_out_tmp.groupby(by=['tags1', 'tags2'], as_index=False)['number'].agg(sum)
    return df_out


In [48]:
## 选择cliques >=3 的互作
import networkx as nx
import pandas as pd
import itertools
def cliques_interaction(input_f):
    graphics = nx.read_edgelist(input_f)
    cliques = nx.find_cliques(graphics)
    list_tmp = []
    for i in cliques:
        if len(i)>2:
            k = sorted(i)
            #print(k)
            one_tmp = list(itertools.combinations(k, 2))
            list_tmp = list_tmp+one_tmp
    df_output = pd.DataFrame(list_tmp)
    df_output.drop_duplicates(keep='first', inplace=True)
    return df_output
    #df_output.to_csv(output_f, sep='\t', index=False)
    

In [59]:
## 分析HiC TAD片段互作 cliques
df_TAD_cliques = cliques_interaction('F:/TM-1_leaf_3D-genome/HiC/TAD/Cliques/method2/TM1_leaf_TAD_node_D01-1.txt')
df_TAD_cliques.columns = ['source', 'target']
df_TAD_weigth = pd.read_csv('F:/TM-1_leaf_3D-genome/HiC/TAD/Cliques/method2/TM1_leaf_TAD_node_D01.txt', sep='\t')
#print(df_TAD_weigth)
df_out = pd.merge(df_TAD_weigth, df_TAD_cliques, how='inner', on=('source', 'target'))
df_out.to_csv('F:/TM-1_leaf_3D-genome/HiC/TAD/Cliques/method2/TM1_leaf_TAD_node_D01_cliques_edg.txt', sep='\t', index=False)

In [61]:
## 分析Pore-C TAD 片段互作cliques
df_TAD_cliques = cliques_interaction('F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/Pore_C_TAD_50_005_node_D01-1.txt')
df_TAD_cliques.columns = ['source', 'target']
df_TAD_weigth = pd.read_csv('F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/Pore_C_TAD_50_005_node_D01.txt', sep='\t')
#print(df_TAD_weigth)
df_out = pd.merge(df_TAD_weigth, df_TAD_cliques, how='inner', on=('source', 'target'))
df_out.to_csv('F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/Pore_C_TAD_50_005_node_D01_cliques_edg.txt', sep='\t', index=False) 

In [97]:
## tad 互作节点最高的度, 以及节点的大小
def cliques_node(input_f, output_f):
    graphics = nx.read_edgelist(input_f)
    cliques = nx.node_clique_number(graphics)
    df = pd.DataFrame.from_dict(cliques, orient='index', columns=['cliques'])
    df = df.reset_index().rename(columns = {'index':'tad_id'})
    df = df.query("cliques>2")
    df_tags = pd.read_csv('F:/TM-1_leaf_3D-genome/HiC/TAD/TADLib/TM-1_TADLib_tad_20K_2M_choosed.bed', sep='\t', names=['c', 's', 'e', 'a'])
    df_tags['mid'] = (df_tags['e']-df_tags['s'])/2 + df_tags['s']
    df_tags['mid'] = df_tags['mid'].astype(int)
    df_tags['mid'] = df_tags['mid'].astype(str)
    df_tags['tad_id'] = df_tags['c'] + '-' + df_tags['mid']
    df_tags['size'] = (df_tags['e'] - df_tags['s'])/1000
    df_tags['size'] = df_tags['size'].astype(int)
    df_tags = df_tags[['tad_id', 'size']].copy()
    #print(df_tags)
    df_out = pd.merge(df, df_tags, how='inner', on=['tad_id'])
    df_out.to_csv(output_f, sep='\t', index=False)

In [98]:
## 对Pore-C 节点打上标签
cliques_node('F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/Pore_C_TAD_50_005_node_A05-1.txt', 'F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/Pore_C_TAD_50_005_node_A05_cliques_node.txt')

In [99]:
## 对Hi-C 的节点打上标签
cliques_node('F:/TM-1_leaf_3D-genome/HiC/TAD/Cliques/method2/TM1_leaf_TAD_node_A05-1.txt', 'F:/TM-1_leaf_3D-genome/HiC/TAD/Cliques/method2/TM1_leaf_TAD_node_A05_cliques_node.txt')

In [22]:
## tad 互作节点最高的度, 以及TAD 的位置
import pandas as pd
import networkx as nx
def cliques_node_TAD(input_f, output_f):
    graphics = nx.read_edgelist(input_f)
    cliques = nx.node_clique_number(graphics)
    #print(cliques)
    df = pd.DataFrame.from_dict(cliques, orient='index', columns=['cliques'])
    df = df.reset_index().rename(columns = {'index':'tad_id'})
    df = df.query("cliques>2")
    df_tags = pd.read_csv('F:/TM-1_leaf_3D-genome/HiC/TAD/TADLib/TM-1_TADLib_tad_20K_2M_choosed.bed', sep='\t', names=['c', 's', 'e', 'a'])
    df_tags['mid'] = (df_tags['e']-df_tags['s'])/2 + df_tags['s']
    df_tags['mid'] = df_tags['mid'].astype(int)
    df_tags['mid'] = df_tags['mid'].astype(str)
    df_tags['tad_id'] = df_tags['c'] + '-' + df_tags['mid']
    df_tags['size'] = (df_tags['e'] - df_tags['s'])/1000
    df_tags['size'] = df_tags['size'].astype(int)
    #df_tags = df_tags[['tad_id', 'size']].copy()
    #print(df_tags)
    df_out = pd.merge(df, df_tags, how='inner', on=['tad_id'])
    df_out.to_csv(output_f, sep='\t', index=False, columns=['tad_id', 'c', 's', 'e', 'cliques'])

In [ ]:
## hic cliques
cliques_node_TAD('F:\TM-1_leaf_3D-genome\HiC\TAD\Cliques\method2\TM1_leaf_TAD_node-1.txt', 'F:\TM-1_leaf_3D-genome\HiC\TAD\Cliques\method2\TM1_leaf_TAD_node_cliques.txt')

In [23]:
## porec cliques
cliques_node_TAD('F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/Pore_C_TAD_50_005_node1.txt', 'F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/Pore_C_TAD_50_005_node_cliques.txt')

In [20]:
df_tad = pd.read_csv('F:\TM-1_leaf_3D-genome\HiC\TAD\Cliques\method2\TM1_leaf_TAD_node_cliques.txt', sep='\t')
df_porec = pd.read_csv('F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/Pore_C_TAD_50_005_node_cliques.txt', sep='\t')
df_tad_porec = df_tad.append(df_porec, ignore_index=True)
df_tad_porec_nodup = df_tad_porec.drop_duplicates(keep='first')
print("PoreC和HiC相同的TAD cliques 数目: {}".format(df_tad_porec.shape[0]-df_tad_porec_nodup.shape[0]))

PoreC和HiC相同的TAD cliques 数目: 227


In [24]:
## 给基因打上cliques 的标签
def gene_cliques(gene_f, cliques_f, output_f):
    df_gene    = pd.read_csv(gene_f, sep='\t')
    df_cliques = pd.read_csv(cliques_f, sep='\t')
    df_result  = pd.DataFrame()
    for index, row in df_cliques.iterrows():
        c, s, e, n = row['c'], row['s'], row['e'], int(row['cliques'])
        df_tmp = df_gene.query("Chr==@c & Start<=@e & End>=@s").copy()
        df_tmp['cliques'] = [n]*df_tmp.shape[0]
        df_result = df_result.append(df_tmp, ignore_index=True)
    df_result.to_csv(output_f, sep='\t',  index=False)

In [30]:
## 计算HiC 不同TAD cliques 所包含基因的表达量
gene_cliques('F:/TM-1_leaf_3D-genome/RNA_Seq/TM-1_allgene_FPKM_TPM.txt', 'F:/TM-1_leaf_3D-genome/HiC/TAD/Cliques/method2/TM1_leaf_TAD_node_cliques.txt', 'F:/TM-1_leaf_3D-genome/HiC/TAD/Cliques/method2/all_TAD_cliques_gene_fpkm_max.txt') 

In [25]:
## 计算PoreC 不同TAD cliques 所包含基因的表达量
gene_cliques('F:/TM-1_leaf_3D-genome/RNA_Seq/TM-1_allgene_FPKM_TPM.txt', 'F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/Pore_C_TAD_50_005_node_cliques.txt', 'F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/Pore_C_TAD_50_005_node_cliques_fpkm.txt') 


In [26]:
## 分析porc 和hic TAD 最大cliques 所能够包含基因的动态变化
import pandas as pd
df_hic = pd.read_csv('F:/TM-1_leaf_3D-genome/HiC/TAD/Cliques/method2/all_TAD_cliques_gene_fpkm_max.txt', sep='\t')
# print(df_hic.shape[0])
df_hic.sort_values(by='cliques', inplace=True)
df_hic.drop_duplicates(keep='last', inplace=True, subset=['Geneid'])
df_porec = pd.read_csv("F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/Pore_C_TAD_50_005_node_cliques_fpkm.txt", sep='\t')
df_porec.drop_duplicates(keep='last', inplace=True, subset=['Geneid'])
df_porec.drop_duplicates(keep='first', inplace=True, subset=['Geneid'])
df_gene = pd.read_csv('F:/TM-1_leaf_3D-genome/RNA_Seq/TM-1_allgene_FPKM_TPM.txt', sep='\t')
df_hic_new = df_hic[['Geneid', 'cliques']].copy()
df_hic_new.columns = ['Geneid', 'hic']
df_porec_new = df_porec[['Geneid', 'cliques']].copy()
df_porec_new.columns = ['Geneid', 'porec']
df_gene = df_gene[['Chr', 'Geneid']].copy()
df_hic_gene = pd.merge(df_gene, df_hic_new, how='left', on=['Geneid'])
df_hic_porec_gene = pd.merge(df_hic_gene, df_porec_new, how='left', on=['Geneid'])
df_need = df_hic_porec_gene.query("Chr.str.contains('Ghir')", engine="python").copy()
df_need.fillna("nocliques", inplace=True)
df_result = df_need.query("~(hic=='nocliques' & porec=='nocliques')")
df_result = df_result.groupby(by=['hic', 'porec'], as_index=False)['Geneid'].count()
print(df_result)
df_nocliques = df_result.query("porec=='nocliques'").copy()
df_number = df_result.query("porec!='nocliques'").copy()
df_number['porec2'] = df_number['porec'].apply(lambda x: ">22" if x>22 else x)
df_result = df_number.append(df_nocliques)
df_result.fillna('nocliques', inplace=True)
df_result['porec'] = df_result['porec2']
df_result.to_csv("F:/TM-1_leaf_3D-genome/Proe-C_HiC/TAD/Cliques/gene/PoreC_HiC_gene_dynamic.txt", sep='\t', index=False, columns=['hic', 'porec', 'Geneid'])

           hic porec  Geneid
0            3     3     403
1            3     4     745
2            3     5     869
3            3     6     963
4            3     7    1457
..         ...   ...     ...
138  nocliques    23      24
139  nocliques    24      16
140  nocliques    25       9
141  nocliques    28       3
142  nocliques    30       4

[143 rows x 3 columns]


In [27]:
#分析TAD cliques 所包含的基因，可考虑重复TAD 具有不同cliques情况
import pandas as pd
import networkx as nx
## 统计TAD 互作的cliques 数目
def Cliques_TAD(input_f):
    graphics = nx.read_edgelist(input_f)
    cliques = nx.find_cliques(graphics)
    df_tmp = pd.DataFrame(columns=['tad_id', 'cliques'])
    for c in cliques:
        clqs = len(c)
        for i in c:
            #print(i, clqs)
            df_tmp.loc[df_tmp.shape[0]] = [i, clqs]
        else:
            continue
    df_tmp.drop_duplicates(keep='first', inplace=True)
    print(df_tmp.shape[0])
    return df_tmp
## 将TADid 与TAD 位置结合起来
def TAD_position(df):
    df_tags = pd.read_csv('F:/TM-1_leaf_3D-genome/HiC/TAD/TADLib/TM-1_TADLib_tad_20K_2M_choosed.bed', sep='\t', names=['c', 's', 'e', 'a'])
    df_tags['mid'] = (df_tags['e']-df_tags['s'])/2 + df_tags['s']
    df_tags['mid'] = df_tags['mid'].astype(int)
    df_tags['mid'] = df_tags['mid'].astype(str)
    df_tags['tad_id'] = df_tags['c'] + '-' + df_tags['mid']
    df_out = pd.merge(df, df_tags, how='inner', on=['tad_id'])
    return df_out

## 分析TAD 所包含的基因，并且给基因打上cliques的标签
# hic
def gene_cliques(df):
    df_gene    = pd.read_csv('F:/TM-1_leaf_3D-genome/RNA_Seq/TM-1_allgene_FPKM_TPM.txt', sep='\t')
    df_cliques = df
    df_result  = pd.DataFrame()
    for index, row in df_cliques.iterrows():
        c, s, e, n = row['c'], row['s'], row['e'], int(row['cliques'])
        df_tmp = df_gene.query("Chr==@c & Start<=@e & End>=@s").copy()
        df_tmp['cliques'] = [n]*df_tmp.shape[0]
        df_result = df_result.append(df_tmp, ignore_index=True)
    return df_result
## porec 
def gene_cliques2(df, df_gene):
    #df_gene    = pd.read_csv('F:/TM-1_leaf_3D-genome/RNA_Seq/TM-1_allgene_FPKM_TPM.txt', sep='\t')
    df_cliques = df
    df_result  = pd.DataFrame()
    for index, row in df_cliques.iterrows():
        c, s, e, n = row['c'], row['s'], row['e'], int(row['cliques'])
        df_tmp = df_gene.query("Chr==@c & Start<=@e & End>=@s").copy()
        df_tmp['cliques'] = [n]*df_tmp.shape[0]
        df_result = df_result.append(df_tmp, ignore_index=True)
    return df_result
## 整理所有脚本
def TAD_cliques_gene(input_f, gene_df, output_f):
    tad_cliques_tag = Cliques_TAD(input_f)
    tad_cliques_position = TAD_position(tad_cliques_tag)
    #tad_cliques_gene = gene_cliques(tad_cliques_position, gene_df) ## Hi-C
    tad_cliques_gene = gene_cliques2(tad_cliques_position, gene_df) ## PoreC
    tad_cliques_gene.drop_duplicates(keep='first', inplace=True)
    tad_cliques_gene.to_csv(output_f, sep='\t', index=False)


In [14]:
## TAD cliques 所包含的基因
TAD_cliques_gene("F:/TM-1_leaf_3D-genome/HiC/TAD/Cliques/method2/TM1_leaf_TAD_node-1.txt", "F:/TM-1_leaf_3D-genome/HiC/TAD/Cliques/method2/all_TAD_cliques_gene_fpkm.txt")

In [34]:
## 分析PoreC 的TAD cliques 所能够包含的基因
##! 将PoreC TAD 互作划分为单条染色体互作， 并根据单条染色体来分析所包含的基因
import pandas as pd
df = pd.read_csv("F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/Pore_C_TAD_50_005_node1.txt", names=['source', 'target'], sep='\t')
df_gene = pd.read_csv('F:/TM-1_leaf_3D-genome/RNA_Seq/TM-1_allgene_FPKM_TPM.txt', sep='\t')
chrs = ['Ghir_A01', 'Ghir_A02', 'Ghir_A03', 'Ghir_A04', 'Ghir_A05', 'Ghir_A06', 'Ghir_A07', 'Ghir_A08', 'Ghir_A09', 'Ghir_A10', 'Ghir_A11', 'Ghir_A12', 'Ghir_A13', 'Ghir_D01', 'Ghir_D02', 'Ghir_D03', 'Ghir_D04', 'Ghir_D05', 'Ghir_D06', 'Ghir_D07', 'Ghir_D08', 'Ghir_D09', 'Ghir_D10', 'Ghir_D11', 'Ghir_D12', 'Ghir_D13']
for c in chrs:
    input_f = "F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/{}.txt".format(c)
    output_f = "F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/{}_cliques_gene.txt".format(c)
    df_chr = df.query("source.str.contains(@c)", engine='python')
    df_chr.to_csv(input_f, sep='\t',index=False, header=False)
    df_gene_chr = df_gene.query("Chr.str.contains(@c)", engine='python')
    #print(df_chr)
    TAD_cliques_gene(input_f, df_gene_chr, output_f)
#! 文件合并
df_tmp = pd.DataFrame()
for c in chrs:
    input_f = "F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/{}_cliques_gene.txt".format(c)
    print(input_f)
    df = pd.read_csv(input_f, sep='\t')
    df_tmp = df_tmp.append(df, ignore_index=True)
df_tmp.drop_duplicates(keep='first', inplace=True)
df_tmp['cliques2'] = df_tmp['cliques'].apply(lambda x: ">22" if x>22 else x)
df_tmp.to_csv("F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/PoreC_cliques_gene.txt", sep='\t', index=False)


F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/Ghir_A01_cliques_gene.txt
F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/Ghir_A02_cliques_gene.txt
F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/Ghir_A03_cliques_gene.txt
F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/Ghir_A04_cliques_gene.txt
F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/Ghir_A05_cliques_gene.txt
F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/Ghir_A06_cliques_gene.txt
F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/Ghir_A07_cliques_gene.txt
F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/Ghir_A08_cliques_gene.txt
F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/Ghir_A09_cliques_gene.txt
F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/Ghir_A10_cliques_gene.txt
F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/Ghir_A11_cliques_gene.txt
F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/Ghir_A12_cliques_gene.txt
F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/Ghir_A13_cliques_gene.txt
F:/TM-1_leaf

In [38]:
#! 分析TAD cliques与A/B compartment 占比间的关系
import pandas as pd
import networkx as nx
def Cliques_TAD(cliques, n_cliques):
    cliques_tad = []
    for c in cliques:
        clqs = len(c)
        if clqs == n_cliques:
            cliques_tad = cliques_tad + c
    df_return = pd.DataFrame({"tad_id":cliques_tad})
    return df_return
def TAD_position(df):
    df_tags = pd.read_csv('F:/TM-1_leaf_3D-genome/HiC/TAD/TADLib/TM-1_TADLib_tad_20K_2M_choosed.bed', sep='\t', names=['c', 's', 'e', 'a'])
    df_tags['mid'] = (df_tags['e']-df_tags['s'])/2 + df_tags['s']
    df_tags['mid'] = df_tags['mid'].astype(int)
    df_tags['mid'] = df_tags['mid'].astype(str)
    df_tags['tad_id'] = df_tags['c'] + '-' + df_tags['mid']
    df_out = pd.merge(df, df_tags, how='inner', on=['tad_id'])
    return df_out


In [8]:
## 分析AB compartment 与TAD 占比关系
import pandas as pd
df = pd.read_csv('F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/ABcpt/PoreC_all_TAD_cliques_abcpt.txt ', sep='\t', names=['c', 's', 'e', 'cliques',  'ABcpt'])
df['size'] = df['e'] - df['s']
df_return = df.groupby(by=['cliques', 'ABcpt'], as_index=False)['size'].sum()
print(df_return)
df_return.to_csv('F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/ABcpt/PoreC_cliques_AB_cpt_ratio.txt', sep='\t', index=False)

    cliques ABcpt        size
0         3  Acpt   186680000
1         3  Bcpt   210540000
2         4  Acpt   274300000
3         4  Bcpt   314700000
4         5  Acpt   360460000
5         5  Bcpt   425540000
6         6  Acpt   418740000
7         6  Bcpt   509700000
8         7  Acpt   465140000
9         7  Bcpt   614640000
10        8  Acpt   510460000
11        8  Bcpt   676180000
12        9  Acpt   546520000
13        9  Bcpt   767660000
14       10  Acpt   562900000
15       10  Bcpt   841940000
16       11  Acpt   572980000
17       11  Bcpt   940220000
18       12  Acpt   542960000
19       12  Bcpt   978620000
20       13  Acpt   473040000
21       13  Bcpt   992880000
22       14  Acpt   439240000
23       14  Bcpt  1016300000
24       15  Acpt   357280000
25       15  Bcpt   993520000
26       16  Acpt   287380000
27       16  Bcpt   931000000
28       17  Acpt   200100000
29       17  Bcpt   829020000
30       18  Acpt   178580000
31       18  Bcpt   740440000
32       1

In [2]:
## 分析每个TAD所能够参与的所有cliques
#分析TAD cliques 所包含的基因，可考虑重复TAD 具有不同cliques情况
import pandas as pd
import networkx as nx
## 统计TAD 互作的cliques 数目
def Cliques_TAD(input_f):
    graphics = nx.read_edgelist(input_f)
    cliques = nx.find_cliques(graphics)
    df_tmp = pd.DataFrame(columns=['tad_id', 'cliques'])
    for c in cliques:
        clqs = len(c)
        for i in c:
            #print(i, clqs)
            df_tmp.loc[df_tmp.shape[0]] = [i, clqs]
        else:
            continue
    df_tmp.drop_duplicates(keep='first', inplace=True)
    print(df_tmp.shape[0])
    return df_tmp
## 将TADid 与TAD 位置结合起来
def TAD_position(df):
    df_tags = pd.read_csv('F:/TM-1_leaf_3D-genome/HiC/TAD/TADLib/TM-1_TADLib_tad_20K_2M_choosed.bed', sep='\t', names=['c', 's', 'e', 'a'])
    df_tags['mid'] = (df_tags['e']-df_tags['s'])/2 + df_tags['s']
    df_tags['mid'] = df_tags['mid'].astype(int)
    df_tags['mid'] = df_tags['mid'].astype(str)
    df_tags['tad_id'] = df_tags['c'] + '-' + df_tags['mid']
    df_out = pd.merge(df, df_tags, how='inner', on=['tad_id'])
    return df_out
chrs = ['Ghir_A01', 'Ghir_A02', 'Ghir_A03', 'Ghir_A04', 'Ghir_A05', 'Ghir_A06', 'Ghir_A07', 'Ghir_A08', 'Ghir_A09', 'Ghir_A10', 'Ghir_A11', 'Ghir_A12', 'Ghir_A13', 'Ghir_D01', 'Ghir_D02', 'Ghir_D03', 'Ghir_D04', 'Ghir_D05', 'Ghir_D06', 'Ghir_D07', 'Ghir_D08', 'Ghir_D09', 'Ghir_D10', 'Ghir_D11', 'Ghir_D12', 'Ghir_D13']
for c in chrs:
    input_f = "F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/{}.txt".format(c)
    output_f = "F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/{}_cliques.txt".format(c)
    TAD_cliques_tags = Cliques_TAD(input_f)
    df_result = TAD_position(TAD_cliques_tags)
    df_result.to_csv(output_f, sep='\t', index=False, header=False, columns = ['c', 's', 'e', 'cliques'])

2336
2348
2132
1727
2127
2431
2012
2094
1625
2297
2537
2223
2101
1493
1524
1125
1187
1375
1376
1214
1392
962
1499
1681
1229
1313


In [5]:
## 合并单条染色体所有的TAD cliques
import pandas as pd
chrs = ['Ghir_A01', 'Ghir_A02', 'Ghir_A03', 'Ghir_A04', 'Ghir_A05', 'Ghir_A06', 'Ghir_A07', 'Ghir_A08', 'Ghir_A09', 'Ghir_A10', 'Ghir_A11', 'Ghir_A12', 'Ghir_A13', 'Ghir_D01', 'Ghir_D02', 'Ghir_D03', 'Ghir_D04', 'Ghir_D05', 'Ghir_D06', 'Ghir_D07', 'Ghir_D08', 'Ghir_D09', 'Ghir_D10', 'Ghir_D11', 'Ghir_D12', 'Ghir_D13']
df_tmp = pd.DataFrame()
for c in chrs:
    input_f =  "F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/{}_cliques.txt".format(c)
    df_chr = pd.read_csv(input_f, sep='\t', names =['c', 's', 'e', 'cliques'])
    df_tmp = df_tmp.append(df_chr, ignore_index=True)
df_tmp = df_tmp.query("cliques>2")
df_tmp.to_csv("F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/PoreC_all_TAD_cliques.txt", sep='\t', index=False, header=False)

In [12]:
## 给cliques 打上标签
chrs = ['Ghir_A01', 'Ghir_A02', 'Ghir_A03', 'Ghir_A04', 'Ghir_A05', 'Ghir_A06', 'Ghir_A07', 'Ghir_A08', 'Ghir_A09', 'Ghir_A10', 'Ghir_A11', 'Ghir_A12', 'Ghir_A13', 'Ghir_D01', 'Ghir_D02', 'Ghir_D03', 'Ghir_D04', 'Ghir_D05', 'Ghir_D06', 'Ghir_D07', 'Ghir_D08', 'Ghir_D09', 'Ghir_D10', 'Ghir_D11', 'Ghir_D12', 'Ghir_D13']
for c in chrs:
    input_f =   "F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/{}_cliques.txt".format(c)
    df_chr = pd.read_csv(input_f, sep='\t', names =['c', 's', 'e', 'cliques'])
    df_chr['tags'] = df_chr['cliques'].apply(lambda x: "a" if 3<=x<=7 else ( "b" if 8<=x<=12 else ("c" if 13<=x<=17 else ("d" if 18<=x<=22 else "e"))))
    output_f = "F:/TM-1_leaf_3D-genome/Pore-C/TAD/TAD_cliques/chr/{}_cliques_tags.txt".format(c)
    df_chr.to_csv(output_f, sep="\t", index=False)